# Extract WMAP instrument parameters

In [ ]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
    f"Instrument model exported from Lambda WMAP DR5",
]

In [ ]:
import numpy as np
from astropy.io import fits

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from pathlib import Path
# Check if the file does not exist before downloading
file_path = Path("wmap_ampl_bl_9yr_v5p1.tar.gz")
if not file_path.exists():
    !wget https://lambda.gsfc.nasa.gov/data/map/dr5/ancillary/beams/wmap_ampl_bl_9yr_v5p1.tar.gz
    !tar xzvf wmap_ampl_bl_9yr_v5p1.tar.gz

In [ ]:
%ls *V1*

In [ ]:
freq_bands = ['K', 'Ka', 'Q', 'V', 'W']

In [ ]:
from pathlib import Path

# Get all files matching the pattern
files = Path('.').glob('wmap_ampl_bl*txt')

# Extract channel names from filenames
channels = [file.stem.split('_')[3] for file in files]

print(channels)

## Create the beam files

In [ ]:
ls *V1*

In [ ]:
!head -n 10 wmap_ampl_bl_V1_9yr_v5p1.txt

In [ ]:
from astropy.table import QTable

beam = {}
for ch in channels:
    wmap_data = np.loadtxt(f'wmap_ampl_bl_{ch}_9yr_v5p1.txt')
    beam[ch] = QTable(data=[wmap_data[:, 0], wmap_data[:, 1]], names=["ell", "B"])
    beam[ch]["B"] /= np.max(beam[ch]["B"])
    beam[ch].meta["comments"] = ipac_warning
    beam[ch].write(f"beam_{ch}.tbl", format="ascii.ipac", overwrite=True)

In [ ]:
for freq_band in freq_bands:
    plt.figure()
    plt.title(f"Beam for {freq_band} band (Log-Log)")
    for ch in channels:
        if ch.startswith(freq_band):
            if freq_band == "K" and ch[1] == "a":
                continue
            beam_data = beam[ch]
            plt.loglog(beam_data["ell"], beam_data["B"], label=f"{ch}")
    plt.legend()
    plt.grid()
    plt.xlabel("Multipole moment (ell)")
    plt.ylabel("Beam response (B)")
    plt.show()


## Create bandpass files

In [ ]:
from pathlib import Path

# Define the file path
file_path = Path("wmap_bandpass_v5.tar.gz")

# Check if the file does not exist before downloading
if not file_path.exists():
    !wget https://lambda.gsfc.nasa.gov/data/map/dr5/ancillary/response/wmap_bandpass_v5.tar.gz
    !tar xzvf wmap_bandpass_v5.tar.gz

In [ ]:
# List all files matching the pattern
bandpass_files = list(Path('.').glob('wmap_bandpass_*.cbp'))

# Extract the V21 part from the filenames
bandpass_channels = [file.stem.split('_')[2] for file in bandpass_files]

print(bandpass_channels)

In [ ]:
!head -n 20 wmap_bandpass_V22_v5.cbp

In [ ]:
# Load the data from the file
bandpass_data = np.loadtxt('wmap_bandpass_V21_v5.cbp')

# Display the first few rows of the data
print(bandpass_data[:5])

In [ ]:
bandpass_data.shape

In [ ]:
for freq_band in freq_bands:
    plt.figure()
    plt.title(f"Bandpass for {freq_band} band (Linear)")
    for ch in bandpass_channels:
        if ch.startswith(freq_band):
            if freq_band == "K" and ch[1] == "a":
                continue
            bandpass_data = np.loadtxt(f'wmap_bandpass_{ch}_v5.cbp')

            # plot bandpass_data[:,1] and 2 against 0
            plt.plot(bandpass_data[:, 0], bandpass_data[:, 1], label=f"{ch} 1")
            plt.plot(bandpass_data[:, 0], bandpass_data[:, 2], label=f"{ch} 2")
    plt.legend()
    plt.grid()
    plt.xlabel("Frequency (GHz)")
    plt.ylabel("Bandpass")
    plt.show()

    plt.figure()
    plt.title(f"Bandpass for {freq_band} band (Semilog)")
    for ch in bandpass_channels:
        if ch.startswith(freq_band):
            if freq_band == "K" and ch[1] == "a":
                continue
            bandpass_data = np.loadtxt(f'wmap_bandpass_{ch}_v5.cbp')

            # plot bandpass_data[:,1] and 2 against 0
            plt.semilogy(bandpass_data[:, 0], bandpass_data[:, 1], label=f"{ch} 1")
            plt.semilogy(bandpass_data[:, 0], bandpass_data[:, 2], label=f"{ch} 2")
    plt.legend()
    plt.grid()
    plt.xlabel("Frequency (GHz)")
    plt.ylabel("Bandpass")
    plt.show()